In [ ]:
from arcgis.gis import GIS
import os
import datetime
# Get the current date and time
current_datetime = datetime.datetime.now()
# Format the date and time as YYYYMMDDhhmmss
formatted_datetime = current_datetime.strftime("%Y%m%d%H%M%S")

In [ ]:
#login parameters:
output_location = r'C:\temp' #LOCATION WHERE OUTPUTS WILL BE SAVED
USERNAME = '' #YOUR_DIEM_USERNAME
PASS = ''# YOUR_PASSWORD
#survey parameters:
iso3 = ''#ISO3 OF THE COUNTRY - empty string for selecting all countries
round = '' # DIEM SURVEY ROUND NUMBER (integer - empty string for selecting all rounds)
#download_parameters
download_microdata = True
download_aggregated = False
#aggregated data parameters:
return_geometry_in_aggregated_data = False #IN CASE download_aggregated = True, DECIDE WHETHER GEOMETRY SHOULD BE DOWNLOADED OR NOT. THIS VARIABLE CAN BE IGNORED IF download_aggregated = False
geometry_export_format = 'File Geodatabase' #IN CASE return_geometry_in_aggregated_data = True, DECIDE THE FORMAT TO EXPORT THE DATA TO. THIS VARIABLE CAN BE IGNORED IF return_geometry_in_aggregated_data = False. ALLLOWED TYPES: Shapefile, CSV, File Geodatabase, Feature Collection, GeoJson, Scene Package, KML, Excel, geoPackage, or Vector Tile Package.

In [ ]:
# Initialize the where clause
where_clause = ""

# Check the conditions and update the where clause accordingly
if iso3 and round:
    where_clause = "adm0_iso3 = '%s' and round = %s" % (iso3, round)
elif iso3:
    where_clause = "adm0_iso3 = '%s'" % iso3
elif round:
    where_clause = "round = %s" % round
else:
    where_clause = "1=1"  # This will select all records if both are empty

In [ ]:
gis = GIS('https://hqfao-hub.maps.arcgis.com/home/group.html?id=8d9585ba920b4e16afe7bfd323d3a6f6&view=list#content', USERNAME, PASS)
# Get the group using the Group ID (From the group URL)
group = gis.groups.get("8d9585ba920b4e16afe7bfd323d3a6f6")
group_items = group.content()
for group_item in group_items:
    print(f"Processing the {group_item.title} dataset.")
    if "microdata" in group_item.title.lower():
        if download_microdata:
            subset_feature_layer = group_item.tables[0]
            #for downloading entire dataset, just remove "where="adm0_iso3 = '%s' and round = %s" % (iso3, round)" from the line below. Note that this may take a long time to complete, especially if geometry is included
            df = subset_feature_layer.query(where=where_clause, out_fields = "*", returnGeometry=False, as_df=True)
            df.to_csv(os.path.join(output_location,group_item.title + "_" + formatted_datetime + ".csv"))
            print(f"Saved the {group_item.title} data in {output_location} .\n")
    else:
         if download_aggregated:
                if return_geometry_in_aggregated_data:
                    subset_feature_layer_id = group_item.layers[0].properties.id
                    parameters = {"layers" :[{"id" : subset_feature_layer_id, "where" : where_clause}]}
                    #for downloading entire dataset, just remove ", parameters = parameters" from the line below. Note that this may take a long time to complete
                    result = group_item.export('{}'.format(group_item.title), geometry_export_format, parameters = parameters)
                    result.download(output_location)
                    # Delete the item from AGOL after it downloads to save space
                    result.delete()
                    print(f"Saved the {group_item.title} data in {output_location} .\n")
                else:
                    subset_feature_layer = group_item.layers[0]
                    #for downloading entire dataset, just remove "where="adm0_iso3 = '%s' and round = %s" % (iso3, round)" from the line below. Note that this may take a long time to complete
                    df = subset_feature_layer.query(where=where_clause, out_fields = "*", returnGeometry=False, as_df=True)
                    df.to_csv(os.path.join(output_location,group_item.title + "_" + formatted_datetime + ".csv"))
                    print(f"Saved the {group_item.title} data in {output_location} .\n")
print("Process completed")